# Feature Engineering

## Arquivos

In [ ]:
import pandas as pd

TRAIN_DIR = "../data/raw/analise-preditiva-de-comportamento-bancario/train.csv"
TEST_DIR = "../data/raw/analise-preditiva-de-comportamento-bancario/test.csv"
SAMPLE_DIR = "../data/raw/analise-preditiva-de-comportamento-bancario/sample_submission.csv"

train = pd.read_csv(TRAIN_DIR)
test = pd.read_csv(TEST_DIR)
sample = pd.read_csv(SAMPLE_DIR)

#print(f"Train shape: {train.shape}")
#print(f"Test shape: {test.shape}")

# Separar target
y_train = train['y']
X_train = train.drop('y', axis=1)

# Concatenar para aplicar transformações uniformemente
df_combined = pd.concat([X_train, test], axis=0, ignore_index=True)
print(f"\nCombined shape: {df_combined.shape}")

## Features

In [ ]:
import pandas as pd
import numpy as np


# 1. Flag de campanha anterior
train['foi_contatado_antes'] = (train['pdays'] != -1).astype(int)

# 2. Meses de alta conversão (seasonality)
high_season_months = ['mar', 'sep', 'oct', 'dec']
train['is_high_season'] = train['month'].isin(high_season_months).astype(int)

# 3. Bins de balance
train['balance_category'] = pd.cut(train['balance'], 
                                 bins=[-np.inf, 0, 1000, 5000, np.inf],
                                 labels=['negative', 'low', 'medium', 'high'])

# 4. Bins de age
train['age_group'] = pd.cut(train['age'], 
                          bins=[0, 25, 35, 50, 65, 100],
                          labels=['jovem', 'adulto_jovem', 'adulto', 'senior', 'aposentado'])

# 5. Intensidade de campanha (normalizado)
train['campaign_intensity'] = train['campaign'] + train['previous']

# 6. Taxa de sucesso histórico (binária)
train['historico_positivo'] = (train['poutcome'] == 'success').astype(int)

# 7. Profissões de alta conversão
high_conversion_jobs = ['student', 'retired', 'unemployed', 'management']
train['job_high_conversion'] = train['job'].isin(high_conversion_jobs).astype(int)

# 8. Tem contato efetivo (não é unknown)
train['has_effective_contact'] = (train['contact'] != 'unknown').astype(int)

# 9. Razão balance/age (capacidade financeira relativa)
train['balance_per_age'] = train['balance'] / (train['age'] + 1)

# 10. Flag de cliente "frio" (muitos contatos sem sucesso)
train['cliente_frio'] = ((train['campaign'] > 3) & (train['poutcome'] != 'success')).astype(int)


## Setup inicial